In [1]:
import optotagging_analysis as oa
import glob
import numpy as np
import re

In [2]:
recording_clipped_folder = glob.glob(f"/data/ecephys_*/**/ecephys_clipped/", recursive=True)
recording_clipped_folder

['/data/ecephys_784956_2025-07-15_15-23-15/ecephys/ecephys_clipped/']

In [3]:
this_recording = recording_clipped_folder[0]
this_session = re.search('\d{6}_\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', this_recording).group(0)
recording_sorted_folder = glob.glob(f'{this_recording[:this_recording.find(this_session)+len(this_session)]}_sorted*')
recording_sorted_folder

['/data/ecephys_784956_2025-07-15_15-23-15_sorted_2025-07-24_03-45-57',
 '/data/ecephys_784956_2025-07-15_15-23-15_sorted-opto_2025-07-18_01-27-50']

In [4]:
analyzer = oa.OptotaggingAnalysis(this_recording, recording_sorted_folder[1])

In [5]:
ap_streams = analyzer.get_stream_names()
this_stream = ap_streams[1]
ap_streams

['ProbeA', 'ProbeB']

In [6]:
timestamps, sorting_outputs, waveform_extractors, extremum_channels = analyzer.get_sorting_output(this_stream)

In [16]:
nidaq_folder = glob.glob(f"/data/ecephys_*/**/ecephys_compressed/experiment1*NI-DAQmx*", recursive=True)[0]

In [19]:
import spikeinterface as si
segment_index = 0
#nidaq_folder = nidaq_folder[0]
#nidq_stream = nidaq_folder.stem
rec_nidaq = si.read_zarr(nidaq_folder)
rec_nidaq = si.split_recording(rec_nidaq)[segment_index]

TypeError: __init__() got an unexpected keyword argument 'dynamic_noise_shaping'

In [17]:
nidaq_folder

'/data/ecephys_784956_2025-07-15_15-23-15/ecephys/ecephys_compressed/experiment1_Record Node 104#NI-DAQmx-103.PXIe-6341.zarr'

In [11]:
all_laser_response_metrics = analyzer.one_probe_laser_responses(this_stream, timestamps, sorting_outputs, waveform_extractors, extremum_channels)

Now processing session 784956_2025-07-15_15-23-15, ProbeA
Channel group 0


100%|██████████| 150/150 [05:58<00:00,  2.39s/it]


Channel group 1


100%|██████████| 143/143 [05:25<00:00,  2.27s/it]


Channel group 2


100%|██████████| 188/188 [06:43<00:00,  2.15s/it]


Channel group 3


100%|██████████| 150/150 [05:52<00:00,  2.35s/it]


In [12]:
responsive_unit_ids = analyzer.get_responsive_neurons(all_laser_response_metrics, sorting_outputs, this_stream)

784956_2025-07-15_15-23-15, ProbeA, shank 0:
0 tagged external_red units: []
0 tagged external_blue units: []
784956_2025-07-15_15-23-15, ProbeA, shank 1:
0 tagged external_red units: []
0 tagged external_blue units: []
784956_2025-07-15_15-23-15, ProbeA, shank 2:
0 tagged external_red units: []
0 tagged external_blue units: []
784956_2025-07-15_15-23-15, ProbeA, shank 3:
0 tagged external_red units: []
0 tagged external_blue units: []


In [9]:
responsive_unit_ids

{0: {'internal_blue': array([  2,  29,  32, 178])}}

In [9]:
import plotting_funcs as pf
from importlib import reload
reload(pf)
for shank in range(4):
    try:
        shank_units = responsive_unit_ids[shank]
        for trial_type in list(shank_units.keys()):
            unit_ids = shank_units[trial_type]
            this_sorting_output = sorting_outputs[shank]
            this_waveform_extractor = waveform_extractors[shank]
            this_laser_response_metrics = all_laser_response_metrics[shank]
            fig_title = f"{this_session}_{this_stream}_shank{shank}_{trial_type.split('_')[-1]}_responsive"

            # for the different ways probe might be named in the trials table...
            probe_name = this_stream
            if probe_name[-3:] == '-AP':
                probe_name = probe_name[:-3]
            if 'emission_location' in analyzer.trial_ids.columns:
                all_emission_locations = np.unique(analyzer.trial_ids.emission_location)
                if probe_name not in all_emission_locations:
                    probe_name = probe_name[:5]+' '+probe_name[-1]
                    if probe_name not in all_emission_locations:
                        for location in all_emission_locations:
                            if probe_name in location:
                                probe_name = location

            trial_types = np.unique(analyzer.trial_ids['type'])

            pf.multi_unit_raster_plot(unit_ids, this_sorting_output, timestamps, this_waveform_extractor, analyzer.trial_ids, analyzer.laser_onset_times, this_laser_response_metrics, trial_types, probe_name, fig_title)
    except KeyError:
        continue
    
    #pf.multi_unit_raster_plot(unit_ids, sorting_output, timestamps, waveform_extractor, event_ids, laser_onset_times, laser_response_metrics, trial_types, session, probe, label,)

In [10]:
analyzer.trial_ids['emission_location']

0      surface (Probe B)
1      surface (Probe A)
2      surface (Probe A)
3      surface (Probe B)
4      surface (Probe B)
             ...        
355    surface (Probe A)
356    surface (Probe B)
357    surface (Probe A)
358    surface (Probe B)
359    surface (Probe B)
Name: emission_location, Length: 360, dtype: object

In [ ]:
# plot all (good?) units to look for responsive ones
import plotting_funcs as pf
from importlib import reload
reload(pf)
for shank in range(4):
    this_sorting_output = sorting_outputs[shank]
    this_waveform_extractor = waveform_extractors[shank]
    all_units = this_sorting_output.unit_ids
    decoder_label = this_sorting_output.get_property('decoder_label')
    unit_ids = all_units[decoder_label != 'noise']
    fig_title = f"{this_session}_{this_stream}_shank{shank}_all_not_noise"

    # for the different ways probe might be named in the trials table...
    probe_name = this_stream
    if probe_name[-3:] == '-AP':
        probe_name = probe_name[:-3]
    if 'emission_location' in analyzer.trial_ids.columns:
        all_emission_locations = np.unique(analyzer.trial_ids.emission_location)
        if probe_name not in all_emission_locations:
            probe_name = probe_name[:5]+' '+probe_name[-1]
            if probe_name not in all_emission_locations:
                for location in all_emission_locations:
                    if probe_name in location:
                        probe_name = location

    trial_types = np.unique(analyzer.trial_ids['type'])

    pf.multi_unit_raster_plot(unit_ids, this_sorting_output, timestamps, this_waveform_extractor, analyzer.trial_ids, analyzer.laser_onset_times, trial_types, probe_name, fig_title)
    
    #pf.multi_unit_raster_plot(unit_ids, sorting_output, timestamps, waveform_extractor, event_ids, laser_onset_times, laser_response_metrics, trial_types, session, probe, label,)

784956_2025-07-15_15-23-15_ProbeB_shank0_all_not_noise.png saved
784956_2025-07-15_15-23-15_ProbeB_shank1_all_not_noise.png saved
784956_2025-07-15_15-23-15_ProbeB_shank2_all_not_noise.png saved


IndexError: list index out of range

> /tmp/ipykernel_15972/2962211230.py(6)<module>()
      4 reload(pf)
      5 for shank in range(4):
----> 6     this_sorting_output = sorting_outputs[shank]
      7     this_waveform_extractor = waveform_extractors[shank]
      8     all_units = this_sorting_output.unit_ids



In [7]:
# plot all noise units to at least see artifacts aligned to stim!!!
import plotting_funcs as pf
from importlib import reload
reload(pf)
for shank in range(4):
    this_sorting_output = sorting_outputs[shank]
    this_waveform_extractor = waveform_extractors[shank]
    all_units = this_sorting_output.unit_ids
    decoder_label = this_sorting_output.get_property('decoder_label')
    unit_ids = all_units[decoder_label == 'noise']
    fig_title = f"{this_session}_{this_stream}_shank{shank}_all_noise"

    # for the different ways probe might be named in the trials table...
    probe_name = this_stream
    if probe_name[-3:] == '-AP':
        probe_name = probe_name[:-3]
    if 'emission_location' in analyzer.trial_ids.columns:
        all_emission_locations = np.unique(analyzer.trial_ids.emission_location)
        if probe_name not in all_emission_locations:
            probe_name = probe_name[:5]+' '+probe_name[-1]
            if probe_name not in all_emission_locations:
                for location in all_emission_locations:
                    if probe_name in location:
                        probe_name = location

    trial_types = np.unique(analyzer.trial_ids['type'])

    pf.multi_unit_raster_plot(unit_ids, this_sorting_output, timestamps, this_waveform_extractor, analyzer.trial_ids, analyzer.laser_onset_times, trial_types, probe_name, fig_title)
    
    #pf.multi_unit_raster_plot(unit_ids, sorting_output, timestamps, waveform_extractor, event_ids, laser_onset_times, laser_response_metrics, trial_types, session, probe, label,)

784956_2025-07-15_15-23-15_ProbeB_shank0_all_noise.png saved
784956_2025-07-15_15-23-15_ProbeB_shank1_all_noise.png saved
784956_2025-07-15_15-23-15_ProbeB_shank2_all_noise.png saved


IndexError: list index out of range

In [24]:
len(sorting_outputs[0].get_property('decoder_label'))

861

In [25]:
len(sorting_outputs[0].unit_ids)

861

In [27]:
len(sorting_outputs[0].unit_ids[sorting_outputs[0].get_property('decoder_label') != 'noise'])

418

In [16]:
template_ext = this_waveform_extractor.get_extension("templates")
template_ext.get_unit_template(1)

array([[ 0.29951999, -0.62829006,  0.02222962, ...,  0.        ,
         0.        ,  0.        ],
       [-0.65519989, -0.13338016, -1.13841105, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.12051018, -0.08658038,  0.55457973, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.49256974, -0.15092972, -0.49257025, ...,  0.        ,
         0.        ,  0.        ],
       [-0.37088987,  0.73007971, -0.93599999, ...,  0.        ,
         0.        ,  0.        ],
       [-0.64934939, -0.05148003, -0.35451028, ...,  0.        ,
         0.        ,  0.        ]])

In [4]:
a = 'external_blue'
a.split('_')[-1]

'blue'